<a href="https://colab.research.google.com/github/Atrib99/ProgrammingAssignment2/blob/master/Bayesian_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import statistics as stats
import numpy as np
from collections import Counter
import re
import plotly.express as exp
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sms = pd.read_csv('https://dq-content.s3.amazonaws.com/433/SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

print("Total number of rows and columns in the SMS dataset: {}".format(sms.shape), '\n',
      "Total number of non-spam messages: {}\n".format(sms['Label'].value_counts()[0]),
      "Total number of spam messages: {}\n".format(sms['Label'].value_counts()[1]),
      "Total percentage of non-spam messages: {}%\n".format(round(sms['Label'].value_counts(normalize=True)[0] * 100, 2)),
      "Total percentage of spam messages: {}%\n".format(round(sms['Label'].value_counts(normalize=True)[1] * 100, 2)))

sms.head()

Total number of rows and columns in the SMS dataset: (5572, 2) 
 Total number of non-spam messages: 4825
 Total number of spam messages: 747
 Total percentage of non-spam messages: 86.59%
 Total percentage of spam messages: 13.41%



,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
sms_dataset = sms.copy()
sms_dataset['SMS'] = sms_dataset['SMS'].str.lower()
sms_dataset['SMS'] = sms_dataset['SMS'].str.replace('[^\w\s]','')

randomized_data = sms_dataset.sample(frac=1, random_state=1) # Shuffling the rows of data
train_num = round(len(randomized_data) * 0.8)
training_set = randomized_data[:train_num].reset_index(drop=True) # Creating the training_set
test_set = randomized_data[train_num:].reset_index(drop=True) # Creating the test_set
print("Number of messages in the Training Set: {}\n".format(training_set.shape[0]),
      "Number of messages in the Test Set: {}\n".format(test_set.shape[0]))

Number of messages in the Training Set: 4458
 Number of messages in the Test Set: 1114



<ipython-input-3-428db2a8e15e>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  sms_dataset['SMS'] = sms_dataset['SMS'].str.replace('[^\w\s]','')


In [4]:
def Naive_Bayesian_parameters(df, message_feature, label_feature):
# Step 1 - Data Cleaning
  message_feature = message_feature.str.lower()
  message_feature = message_feature.str.replace('[^\w\s]','')

# Step 2 - Calculating our 5 Constants
  print("Bayesian Constants", '\n')
  p_ham = label_feature.value_counts(normalize=True)[0]
  print("P(non-spam message): {}\n".format(p_ham))

  p_spam = label_feature.value_counts(normalize=True)[1]
  print("P(spam message): {}\n".format(p_spam))

  n_ham = label_feature.value_counts()[0]
  print("N(non-spam message): {}\n".format(n_ham))

  n_spam = label_feature.value_counts()[1]
  print("N(spam message): {}\n".format(n_spam))
  alpha = 1

# Counting our individual vocab words
  message_feature = message_feature.str.split()
  full_vocab = []
  for sentence in message_feature:
      for word in sentence:
          full_vocab.append(word)
  set_vocab = list(set(full_vocab))
  n_vocab = len(set_vocab)
  print("N(vocabulary): {}\n".format(n_vocab))


# Step 3A - Creating our parameters for spam messages
  spam_messages = df[df['Label'] == "spam"]
  spam_message_feature = spam_messages['SMS'].str.lower()
  spam_message_feature = spam_message_feature.str.replace('[^\w\s]','')
  spam_message_feature = spam_message_feature.str.split()

  spam_full_vocab = []
  for sentence in spam_message_feature:
    for word in sentence:
      spam_full_vocab.append(word)

  set_spam_vocab = list(set(spam_full_vocab))
  n_words_given_spam = dict(Counter(spam_full_vocab))

  p_word_given_spam_parameters = {word: 1 for word in set_spam_vocab}
  for word in set_spam_vocab:
    p_word_given_spam = ((n_words_given_spam[word] + alpha) / (n_spam + alpha * n_vocab))
    p_word_given_spam_parameters[word] *= p_word_given_spam

# Step 3B - Creating our parameters for non-spam messages
  nonspam = df[df['Label'] == "ham"]
  nonspam_message_feature = nonspam['SMS'].str.lower()
  nonspam_message_feature = nonspam_message_feature.str.replace('[^\w\s]','')
  nonspam_message_feature = nonspam_message_feature.str.split()

  nonspam_full_vocab = []
  for sentence in nonspam_message_feature:
    for word in sentence:
      nonspam_full_vocab.append(word)

  set_nonspam_vocab = list(set(nonspam_full_vocab))
  n_words_given_nonspam = dict(Counter(nonspam_full_vocab))

  p_word_given_nonspam_parameters = {word: 1 for word in set_nonspam_vocab}
  for word in set_nonspam_vocab:
    p_word_given_nonspam = (n_words_given_nonspam[word] + alpha) / (n_ham + alpha * n_vocab)
    p_word_given_nonspam_parameters[word] *= p_word_given_nonspam

  return n_words_given_spam, n_words_given_nonspam, p_word_given_spam_parameters, p_word_given_nonspam_parameters, p_ham, p_spam
